In [98]:
#!pip install geopandas
#!pip install openpyxl
#!pip install folium

In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from geopandas import GeoDataFrame
from openpyexcel import load_workbook
import numpy as np

import folium 
from folium import Circle,Choropleth,Marker
from folium.plugins import HeatMap,MarkerCluster

In [3]:
full_workbook=load_workbook('df.xlsx')
print(full_workbook.sheetnames)

['CHANDRAPUR CIRCLE', 'BALLARSHAH', 'CHANDRAPUR DIV', 'WARORA', 'ALLAPALLI', 'BRAMHAPURI', 'GADCHIROLI']


In [4]:
chandrapur_div_df=pd.read_excel('df.xlsx',sheet_name='CHANDRAPUR DIV')
chanda_district_df=pd.read_excel('df.xlsx',sheet_name='CHANDRAPUR CIRCLE')
chanda_district_df['SECTION_NAME'].value_counts()

Pombhurna DC          171
SHANKARPUR D/C        169
RAJURA DC-II          137
Gondpipri DC-II       130
MADHELI DC            127
                     ... 
Etapalli                2
Kasansoor               1
DC-III, Ballarshah      1
Dhanora DC              1
Gatta                   1
Name: SECTION_NAME, Length: 80, dtype: int64

In [5]:
cate_col=chandrapur_div_df.select_dtypes(exclude=np.number)
cate_col.columns

Index(['REGION', 'ZONE', 'CIRCLE', 'DIVISION', 'SUB_DIVISION', 'SECTION_NAME',
       'APPLICATION DATE', 'CONSUMER NAME', 'ADDRESS', 'DTC', 'SUPPLY DATE',
       'SCHEME NAME', 'APPLN SOURCE', 'METER SOURCE'],
      dtype='object')

In [6]:
chandrapur_div_df_sorted=chandrapur_div_df[['DIVISION','SUB_DIVISION','SECTION_NAME','CONSUMER NAME','ADDRESS','MOBILE','METER NO','LATITUDE','LONGITUDE']]
chanda_district_df_sorted=chanda_district_df[['DIVISION','SUB_DIVISION','SECTION_NAME','CONSUMER NAME','ADDRESS','MOBILE','METER NO','LATITUDE','LONGITUDE']]

In [7]:
for val in chandrapur_div_df_sorted['SECTION_NAME'].value_counts().index:
    print(val)

Ghugus DC
Saoli
Mul Rural
Vyahad D/C
Rural DC
Mul Urban
MIDC DC


In [8]:
df_test=chandrapur_div_df.groupby('SECTION_NAME')[['LATITUDE','LONGITUDE']].value_counts()
df_test

SECTION_NAME  LATITUDE   LONGITUDE
Ghugus DC     20.023503  79.250703    2
              20.010057  79.169470    2
              0.000000   0.000000     1
              19.904987  79.204106    1
              20.011080  79.168204    1
                                     ..
Vyahad D/C    19.976080  79.891447    1
              19.975542  79.892585    1
              19.975538  79.892847    1
              19.972642  79.916285    1
              20.129868  79.882982    1
Length: 380, dtype: int64

In [9]:
#converting the df into gdf
geometry=[Point(xy)for xy in zip(chandrapur_div_df_sorted['LONGITUDE'],chandrapur_div_df_sorted['LATITUDE'])]
geometry2=[Point(xy)for xy in zip(chanda_district_df_sorted['LONGITUDE'],chanda_district_df_sorted['LATITUDE'])]
gdf=GeoDataFrame(chandrapur_div_df_sorted,geometry=geometry)
gdf2=GeoDataFrame(chanda_district_df_sorted,geometry=geometry2)


In [10]:
m_1=folium.Map(location=[20.2095, 79.5603],tiles='openstreetmap',zoom_start=10)
mc=MarkerCluster()


        

for idx, row in gdf2.iterrows():
    #folium.Marker(location=[row['LATITUDE'],row['LONGITUDE']]).add_to(m_1)
    
    mc.add_child(Marker([row['LATITUDE'], row['LONGITUDE']]))

m_1.add_child(mc)
#m_1.save('chanda_divison.html')
m_1


In [11]:
m_2=folium.Map(location=[19.9792146, 79.3031043],tiles='openstreetmap',zoom_start=10)
mc=MarkerCluster()

for idx, row in gdf.iterrows():
    #folium.Marker(location=[row['LATITUDE'],row['LONGITUDE']]).add_to(m_1)
    mc.add_child(Marker([row['LATITUDE'], row['LONGITUDE']],popup=row['CONSUMER NAME']+ ',' +row['ADDRESS'] +','+ str(row['MOBILE'])))


m_2.add_child(mc)
#m_1.save('chanda_divison.html')
m_2




In [19]:
gdf[gdf['SUB_DIVISION']=='CHANDRAPUR S/DN.'].sort_values('LATITUDE')

,DIVISION,SUB_DIVISION,SECTION_NAME,CONSUMER NAME,ADDRESS,MOBILE,METER NO,LATITUDE,LONGITUDE,geometry
99,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,SADHU VITHOBA MUSALE,18/4 MORVA MORVA,7.350127e+09,3732964.0,0.000000,0.000000,POINT (0.00000 0.00000)
51,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,VITHHAL ZOLBAJI BHOYAR,SR. NO. 717 VG PIPARI DIST CHANDRAPUR,9.850402e+09,3732968.0,19.904987,79.204106,POINT (79.20411 19.90499)
71,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,RAJKUMAR VITHOBA NAGPURE,"SR.NO. 234 VG WENDALI TAL, DIST CHANDRAPUR",8.329062e+09,10018976.0,19.905074,79.204760,POINT (79.20476 19.90507)
53,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,REKHA SHANKAR KAMBALE,SR.NO 286 VG- SIDUR DIST CHANDRAPUR,9.767151e+09,3732779.0,19.905312,79.203886,POINT (79.20389 19.90531)
62,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,MURLIDHAR JAIRAM NANDEKAR,"SR.NO. 378 VG SIDUR TAL, DIST CHANDRAPUR",9.604155e+09,10018972.0,19.905391,79.205880,POINT (79.20588 19.90539)
...,...,...,...,...,...,...,...,...,...,...
17,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,VANWASU RUSHI JUNGHARE,108/2 CHICHOLI CHANDRAPUR,9.405155e+09,10018610.0,20.519490,79.162870,POINT (79.16287 20.51949)
12,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,SHRI HANUMAN RUSHI ASUTKAR,121/2 CHICHOLI CHICHOLI CHANDRAPUR,9.623861e+09,10018384.0,20.525910,79.163323,POINT (79.16332 20.52591)
18,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,MANGESH NILKANTH ASUTKAR,121/3 CHICHOLI CHANDRAPUR,1.000000e+10,10018443.0,20.525910,79.163323,POINT (79.16332 20.52591)
15,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,PANDURANG NAMDEV UGHE,112/4 CHICHOLI CHANDRAPUR,1.000000e+10,10018386.0,20.615900,79.161215,POINT (79.16121 20.61590)


In [195]:
chandrapur_div_df_sorted['SUB_DIVISION'].unique()

array(['CHANDRAPUR-II(U)SDN', 'CHANDRAPUR S/DN.', 'MUL S/DN.',
       'SAOLI S/DN'], dtype=object)

In [25]:
#gdf_temp.drop(index=99,inplace=True)
gdf_temp.sort_values('LATITUDE')

,DIVISION,SUB_DIVISION,SECTION_NAME,CONSUMER NAME,ADDRESS,MOBILE,METER NO,LATITUDE,LONGITUDE,geometry
51,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,VITHHAL ZOLBAJI BHOYAR,SR. NO. 717 VG PIPARI DIST CHANDRAPUR,9.850402e+09,3732968.0,19.904987,79.204106,POINT (79.20411 19.90499)
71,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,RAJKUMAR VITHOBA NAGPURE,"SR.NO. 234 VG WENDALI TAL, DIST CHANDRAPUR",8.329062e+09,10018976.0,19.905074,79.204760,POINT (79.20476 19.90507)
53,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,REKHA SHANKAR KAMBALE,SR.NO 286 VG- SIDUR DIST CHANDRAPUR,9.767151e+09,3732779.0,19.905312,79.203886,POINT (79.20389 19.90531)
62,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,MURLIDHAR JAIRAM NANDEKAR,"SR.NO. 378 VG SIDUR TAL, DIST CHANDRAPUR",9.604155e+09,10018972.0,19.905391,79.205880,POINT (79.20588 19.90539)
59,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,EKNATH NATTHU NIKHADE,SR.NO. 14 VG SIDUR GHUGHUS DC DIST CHANDRAPUR,7.875108e+09,3732772.0,19.905801,79.202065,POINT (79.20206 19.90580)
...,...,...,...,...,...,...,...,...,...,...
17,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,VANWASU RUSHI JUNGHARE,108/2 CHICHOLI CHANDRAPUR,9.405155e+09,10018610.0,20.519490,79.162870,POINT (79.16287 20.51949)
12,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,SHRI HANUMAN RUSHI ASUTKAR,121/2 CHICHOLI CHICHOLI CHANDRAPUR,9.623861e+09,10018384.0,20.525910,79.163323,POINT (79.16332 20.52591)
18,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,MANGESH NILKANTH ASUTKAR,121/3 CHICHOLI CHANDRAPUR,1.000000e+10,10018443.0,20.525910,79.163323,POINT (79.16332 20.52591)
15,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,PANDURANG NAMDEV UGHE,112/4 CHICHOLI CHANDRAPUR,1.000000e+10,10018386.0,20.615900,79.161215,POINT (79.16121 20.61590)


In [27]:

from folium.plugins import AntPath
gdf_temp=gdf[gdf['SUB_DIVISION']=='CHANDRAPUR S/DN.'].sort_values('LATITUDE')
gdf_temp.drop(index=99,inplace=True)
pathLatLngs=[(x,y) for x,y in zip(gdf_temp['LATITUDE'],gdf_temp['LONGITUDE'])]

m_3=folium.Map(location=[20.2095, 79.5603],tiles='openstreetmap',zoom_start=10)
mc=MarkerCluster()



# folium.Circle(radius=30000,
#               location=[19.9641678, 79.2752267],
#               color='yellow',
#               weight=5).add_to(m_3)

AntPath(pathLatLngs,delay=4000,weight=3,color='red').add_to(m_3)       

for idx, row in gdf_temp.iterrows():
    #folium.Marker(location=[row['LATITUDE'],row['LONGITUDE']]).add_to(m_1)
    
    mc.add_child(Marker([row['LATITUDE'], row['LONGITUDE']],popup=row['CONSUMER NAME']+ ',' +row['ADDRESS'] +','+ str(row['MOBILE'])))

m_3.add_child(mc)
#m_1.save('chanda_divison.html')
#m_3.save('SAOLI SDN.html')
m_3


In [198]:
gdf

,DIVISION,SUB_DIVISION,SECTION_NAME,CONSUMER NAME,ADDRESS,MOBILE,METER NO,LATITUDE,LONGITUDE,geometry
0,CHANDRAPUR DIVSION,CHANDRAPUR-II(U)SDN,MIDC DC,KAWADU LATARI ASUTKAR,SR.NO. 76/1 VG PADOLI OLD TAL DIST CHANDRAP...,8.411931e+09,3732639.0,19.964466,79.259874,POINT (79.25987 19.96447)
1,CHANDRAPUR DIVSION,CHANDRAPUR-II(U)SDN,MIDC DC,SHOBHA Shaliram Sadafale,s. no. 163 datala datala,9.850316e+09,3732637.0,19.963583,79.261096,POINT (79.26110 19.96358)
2,CHANDRAPUR DIVSION,CHANDRAPUR-II(U)SDN,MIDC DC,MEENABAI NANAJI ITANKAR,SR.NO.979/2 INFRONT OF PODDAR HIGH SCHOOL VG...,7.410577e+09,3732638.0,19.962301,79.256849,POINT (79.25685 19.96230)
3,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,SHANKAR PAIKAJEE GURNULE,213 CHORGAON CHANDRAPUR,9.767173e+09,10018398.0,20.015932,79.255771,POINT (79.25577 20.01593)
4,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,BANDU RAMCHANDRA AWARI,SRY NO 04 AMBHORA CHANDRAPUR,9.011289e+09,10018798.0,20.020494,79.266848,POINT (79.26685 20.02049)
...,...,...,...,...,...,...,...,...,...,...
379,CHANDRAPUR DIVSION,SAOLI S/DN,Vyahad D/C,THIVRU SUKURAM BHURSE,94 KERODA KERODA SAOLI,8.806598e+09,3732031.0,20.070448,79.887910,POINT (79.88791 20.07045)
380,CHANDRAPUR DIVSION,SAOLI S/DN,Vyahad D/C,LIYAKAT SIRAJ SHAIKH,700 MOKHALA MOKHALA SAOLI,9.595285e+09,3732989.0,20.118293,79.899315,POINT (79.89932 20.11829)
381,CHANDRAPUR DIVSION,SAOLI S/DN,Vyahad D/C,SHRIKRUSHNA LAXMAN BHURSE,646 HARAMBA HARAMBA SAOLI,9.158778e+09,3732035.0,19.990635,79.919555,POINT (79.91956 19.99064)
382,CHANDRAPUR DIVSION,SAOLI S/DN,Vyahad D/C,NAMDEV BABURAO WASEKAR,526 SAMDA BUJ SAMDA BUJ SAOLI,8.975600e+09,3732983.0,20.121165,79.949378,POINT (79.94938 20.12117)
